In [623]:
import numpy as np
from PIL import Image
import glob
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split

In [624]:
# Define constants
IMG_SHAPE = (64, 64)
# Download Olivetti faces dataset
olivetti = fetch_olivetti_faces()
X = olivetti.images
y = olivetti.target
# Print info on shapes and reshape where necessary
X = X.reshape((400, 4096))
indices = np.arange(len(X))
Xtrain, Xtest, ytrain, ytest, idx_train, idx_test = train_test_split(X, y, indices)

[20  3 37 16 32 24 30 36  9  7 13 32 13 21 14 17 35 24  4 34  3 20  1 19
 14 39 27 18 39  3 33 17 22  0  1 27  0 27  4  1 39  3 18 31 31 16  3 10
  1 29  6 12 11  2 19 23 34 36 30 39 14 32 22  2 12 26 16 26 16 17 17 17
 22 19 32  2  9 17 36 17 39 34 34  3  5 26 28 32 35 34 28 37 28 10 34 17
 15 27 30 21 35 32 34 33 36 13 14 12 14 38  5 33 28 38 26  9  7 38 15 30
 36  7 31  8 12 11 12 25 18 21 10  4 12 27  2  2 28 27 37 36 30  1 20 38
 19 22 29  9 13 36 23  3 36 19 21  0 10 35  4 23 10 11 35 21  2  2 19 33
 36 33 12 38 29  4 16 29 29 29  0  1 22 31 11 24 16 19 10  4  8 26 37  5
 15 15  8  9 15 22 26 21 31 20 14 38 25  5 35  8  0 23 32  9  6 25 25 35
 23  0 26 25  8  3 15 22  5 25  0  4 18 29  9  6 23 37 31  5 24  8  4 14
 24 16  2 13 35 21 16  1 13 32 24 27  0  5 32 23  6 14 11 20  2 33  7  3
  8 37 11 31  6 25 32  7  0 31 16 26 38 10 12 28 26  1 27 18 27 13  2 39
 38 34 24 21 11 19 10 25 11  7  8 27]
300 300


In [625]:
def crop_image(img):
    width, height = img.size

    new_width = IMG_SHAPE[0]
    new_height = IMG_SHAPE[1]
    left = (width - new_width)/2 - .5
    top = (height - new_height)/2 - .5
    right = (width + new_width)/2 - .5
    bottom = (height + new_height)/2 - .5

    # Crop the center of the image
    im = img.crop((left, top, right, bottom))

    return im

def img_to_vector(path_to_img):
    # Load the image
    img = Image.open(path_to_img).convert('L')
    img = crop_image(img)

    # Convert the image into n²*1 array
    arr = np.array(img)
    flat_array = arr.ravel()

    return flat_array


def vector_to_img(vector, shape = IMG_SHAPE):
    array = vector.reshape(shape)
    img = Image.fromarray(array)
    return img

def sum_of_vectors(arr: []):
    sum_vector = np.zeros(len(arr[0]))
    for i in range(0, len(arr[0])):
        for v in arr:
            sum_vector[i] += v[i]

    return sum_vector


def scalar_multiply_vector(scalar, v):
    arr = np.array([])
    for x in v:
        arr = np.append(arr, x * scalar)
    return arr


def negative_vector(v):
    new_v = []
    for x in v:
        new_v.append(-x)
    return np.array(new_v)

In [626]:
all_imgs = []
training_set_photo_names = []

# Create an np.array from the vectors
training_set = Xtrain

In [627]:
# Average face using numpy
avg_face = training_set.mean(axis=0)

In [628]:
# Let's create the matrix A by subtracting the average face from each face in the training set
A = []
neg_avg_face = negative_vector(avg_face)
sub = None
for v in training_set:
    sub = np.subtract(v, avg_face)
    A.append(sub)

# Convert A to a matrix
A_m = np.asmatrix(A)

A_t = np.array(A).transpose()

In [629]:
# Form the covariance matrix
cov_matrix = np.cov(np.array(A))

In [630]:
# Calculate the eigenvectors of the covariance matrix
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]

In [631]:
# Choose the 10 eigenvectors with the highest eigenvalues as the eigenfaces
eigenfaces = np.array(eigvectors_sort[:20]).transpose()

In [632]:
# Calculate eiganfaces
# Create reduced eigenface space
proj_data = np.dot(training_set.transpose(), eigenfaces)
proj_data = proj_data.transpose()
# Calculate weights for eigenfaces
w = np.array([np.dot(proj_data,i) for i in np.array(A)])

In [633]:
from sklearn.metrics import classification_report

correct_ids = []
predicted_ids = []

# Get images from dataset and convert them to vectors
test_index = 20
for test_index in range(len(Xtest)):
    unknown_face_vector = Xtest[test_index]
    mean_unknown_face = np.subtract(unknown_face_vector, avg_face)
    w_unknown = np.dot(proj_data, mean_unknown_face)
    difference_vector = w - w_unknown
    norms = np.linalg.norm(difference_vector, axis=1)
    index = np.argmin(norms)

    correct_ids.append(y[idx_test[test_index]])
    predicted_ids.append(y[idx_train[index]])

In [634]:
print(classification_report(correct_ids, predicted_ids, zero_division=0))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00         2
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         3
           6       0.60      0.60      0.60         5
           7       1.00      0.50      0.67         4
           8       1.00      1.00      1.00         2
           9       1.00      0.33      0.50         3
          10       1.00      1.00      1.00         2
          11       1.00      1.00      1.00         2
          12       0.33      0.50      0.40         2
          13       1.00      1.00      1.00         3
          14       1.00      1.00      1.00         2
          15       0.50      0.50      0.50         4
          16       0.50      1.00      0.67         1
          17       1.00    